In [66]:
import math
import numpy as np
import pandas as pd

In [78]:
# Training and test data files
train_file = '../input/train.csv'
test_file = '../input/test.csv'
model_file = '../output/titanic.model.json'
model_weights_file = '../output/titanic.model.best.hdf5'
pred_file = '../output/gender_submission.csv'

In [87]:
# Load training data
df_train_raw = pd.read_csv(train_file)
print(df_train_raw.shape)
df_train_raw.info()
df_train_raw.head()

(891, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [88]:
# Prepare the data for training and testing
from sklearn.preprocessing import MinMaxScaler

max_name_len = df_train_raw.Name.map(len).max()
max_ticket_len = df_train_raw.Ticket.map(len).max()

title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev', 'Dr', 'Ms', 'Mlle',
            'Col', 'Capt', 'Mme', 'Countess', 'Don', 'Jonkheer']

import string
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if substring in big_string:
            return substring
    return np.nan

def prep_data(frame, mode='test'):
    # Fill missing Age data with median 
    frame['Age'] = frame['Age'].fillna(frame['Age'].median())
    
    # Generate data about whether adult or minor
    frame['Adult_Or_Minor'] = frame.apply(lambda row: 0 if row['Age'] < 18 else 1, axis=1)

    # Generate data about whether senior citizen
    frame['Senior_Citizen'] = frame.apply(lambda row: 0 if row['Age'] > 65 else 1, axis=1)

    # Fill missing Fare data with median
    frame['Fare'] = frame['Fare'].fillna(frame['Fare'].median())
    
    # Creating new family_size and fare per person columns 
    frame['Family_Size'] = frame['SibSp'] + frame['Parch'] + 1
    frame['Alone'] = frame.apply(lambda row: 1 if row['Family_Size'] == 1 else 0, axis=1)
    frame['Fare_Per_Person'] = frame['Fare']/frame['Family_Size']

    # Convert Sex to number
    frame['Sex'] = pd.Categorical(frame['Sex']).codes

    # Generate data for missing Embarked and convert to number
    frame['Embarked'] = frame['Embarked'].fillna('S')
    frame['Embarked'] = pd.Categorical(frame['Embarked']).codes
    
    # Extract title from name
    frame['Title'] = frame['Name'].map(lambda x: substrings_in_string(x, title_list))
    frame['Title'] = pd.Categorical(frame['Title']).codes

    # Convert Name into characters
    frame['Name_Length'] = frame.apply(lambda row: len(row['Name']), axis=1)
    frame['Words_In_Name'] = frame.apply(lambda row: len(row['Name'].split()), axis=1)    
    for i in range(0, max_name_len):
        col_name = 'Name' + str(i)
        frame[col_name] = frame['Name'].str[i]
        frame[col_name] = frame.apply(lambda row: 0 if isinstance(row[col_name], float) and math.isnan(row[col_name]) else ord(row[col_name]), axis=1)
    frame.pop('Name')    
    
    # Convert Ticket into characters
    frame['Ticket_Length'] = frame.apply(lambda row: len(row['Ticket']), axis=1)
    for i in range(0, max_ticket_len):
        col_name = 'Ticket' + str(i)
        frame[col_name] = frame['Ticket'].str[i]
        frame[col_name] = frame.apply(lambda row: 0 if isinstance(row[col_name], float) and math.isnan(row[col_name]) else ord(row[col_name]), axis=1)
    frame.pop('Ticket')    
    
    # Convert Cabin column to whether in cabin
    frame['Cabin'] = frame['Cabin'].fillna('')
    frame['In_Cabin'] = frame.apply(lambda row: 1 if row['Cabin'] != '' else 0, axis=1)
    frame['Number_Of_Cabins'] = frame.apply(lambda row: len(row['Cabin'].split()), axis=1)    
    frame.pop('Cabin')
    
    frame.fillna(0, axis=1)
    
    # Introduce rows with some noise
    if (mode == 'augment'):
        print('Adding more rows to training data')
        row_count = frame.shape[0]
        print('Row count before: ', row_count)
        col_std = np.std(frame) 
        for i in range(0, row_count):
            row1 = pd.Series(frame.iloc[i])
            row2 = pd.Series(frame.iloc[i])
            col_list = frame.columns.tolist()
            col_list.remove('PassengerId')
            col_list.remove('Survived')
            for col in frame.columns.tolist():
                row1[col] = row1[col] + col_std[col]
                row2[col] = row2[col] - col_std[col]
            if np.random.random_sample() < 0.33:
                frame = frame.append(row1)
            if np.random.random_sample() > 0.66:
                frame = frame.append(row2)
        row_count = frame.shape[0]
        print('Row count after: ', row_count)
    
    
    print("Before scaling: ")
    print(frame.head())
    
    # Scale everything except PassengerId
    min_max_scaler = MinMaxScaler()
    col_list = frame.columns.tolist()
    col_list.remove('PassengerId')
    frame = frame[col_list]
    np_scaled = min_max_scaler.fit_transform(frame)
    frame = pd.DataFrame(np_scaled)
    
    print("After scaling: ")
    print(frame.head())

    return frame



In [89]:
# Prep training data
df_train = prep_data(df_train_raw, mode='train')

# Construct the X array
X_train = np.array(df_train)[:,1:]
X_train = X_train.astype('float32')
print(X_train.shape)
print(X_train[0])

Before scaling: 
   PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  \
0            1         0       3    1  22.0      1      0   7.2500         2   
1            2         1       1    0  38.0      1      0  71.2833         0   
2            3         1       3    0  26.0      0      0   7.9250         2   
3            4         1       1    0  35.0      1      0  53.1000         2   
4            5         0       3    1  35.0      0      0   8.0500         2   

   Adult_Or_Minor        ...         Ticket10  Ticket11  Ticket12  Ticket13  \
0               1        ...                0         0         0         0   
1               1        ...                0         0         0         0   
2               1        ...               49        48        49        50   
3               1        ...                0         0         0         0   
4               1        ...                0         0         0         0   

   Ticket14  Ticket15  Tick

In [90]:
# Extract survived data as predictions
from keras.utils.np_utils import to_categorical
y_train = np.array(df_train)[:,0]
y_train = to_categorical(y_train, 2)
print(y_train.shape)
print(y_train[0:5])

(891, 2)
[[ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]]


In [91]:
# Build a training network

from keras.models import Sequential
from keras.layers import Dense, Dropout, RepeatVector, Flatten, Activation
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD
from keras.layers.advanced_activations import LeakyReLU

model = Sequential()
model.add(Dense(891, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.25))
model.add(Dense(445, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 891)               106029    
_________________________________________________________________
dropout_16 (Dropout)         (None, 891)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 445)               396940    
_________________________________________________________________
dropout_17 (Dropout)         (None, 445)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 2)                 892       
Total params: 503,861
Trainable params: 503,861
Non-trainable params: 0
_________________________________________________________________


In [92]:
# Save the model
model_json = model.to_json()
with open(model_file, 'w') as json_file:
    json_file.write(model_json)

In [93]:
# Train the model
checkpointer = ModelCheckpoint(filepath=model_weights_file, verbose=1, save_best_only=True)
stopper = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=1, mode='auto')
hist = model.fit(X_train, y_train, epochs=100, batch_size=20, validation_split=0.3,
                 callbacks=[checkpointer, stopper], 
                 verbose=1, shuffle=True)

Train on 623 samples, validate on 268 samples
Epoch 1/100
623/623 [==============================] - 0s - loss: 0.6822 - acc: 0.5586 - val_loss: 0.6505 - val_acc: 0.6530
Epoch 2/100
623/623 [==============================] - 0s - loss: 0.6649 - acc: 0.6100 - val_loss: 0.6316 - val_acc: 0.6455
Epoch 3/100
623/623 [==============================] - 0s - loss: 0.6504 - acc: 0.6276 - val_loss: 0.6167 - val_acc: 0.6511
Epoch 4/100
623/623 [==============================] - 0s - loss: 0.6384 - acc: 0.6276 - val_loss: 0.6053 - val_acc: 0.6828
Epoch 5/100
623/623 [==============================] - 0s - loss: 0.6246 - acc: 0.6573 - val_loss: 0.5956 - val_acc: 0.7146
Epoch 6/100
623/623 [==============================] - 0s - loss: 0.6155 - acc: 0.6677 - val_loss: 0.5848 - val_acc: 0.7220
Epoch 7/100
623/623 [==============================] - 0s - loss: 0.6083 - acc: 0.6709 - val_loss: 0.5762 - val_acc: 0.7313
Epoch 8/100
623/623 [==============================] - 0s - loss: 0.6033 - acc: 0.6814

623/623 [==============================] - 0s - loss: 0.5093 - acc: 0.7681 - val_loss: 0.4799 - val_acc: 0.7817
Epoch 28/100
623/623 [==============================] - 0s - loss: 0.4963 - acc: 0.7753 - val_loss: 0.4580 - val_acc: 0.7836
Epoch 29/100
623/623 [==============================] - 0s - loss: 0.4872 - acc: 0.7761 - val_loss: 0.4558 - val_acc: 0.7817
Epoch 30/100
623/623 [==============================] - 0s - loss: 0.4893 - acc: 0.7833 - val_loss: 0.4510 - val_acc: 0.7854
Epoch 31/100
623/623 [==============================] - 0s - loss: 0.4915 - acc: 0.7673 - val_loss: 0.4511 - val_acc: 0.7910
Epoch 32/100
623/623 [==============================] - 0s - loss: 0.4834 - acc: 0.7809 - val_loss: 0.4453 - val_acc: 0.7910
Epoch 33/100
623/623 [==============================] - 0s - loss: 0.4857 - acc: 0.7777 - val_loss: 0.4434 - val_acc: 0.7929
Epoch 34/100
623/623 [==============================] - 0s - loss: 0.4825 - acc: 0.7913 - val_loss: 0.4479 - val_acc: 0.7929
Epoch 35/100


623/623 [==============================] - 0s - loss: 0.4425 - acc: 0.8034 - val_loss: 0.4152 - val_acc: 0.8060
Epoch 57/100
623/623 [==============================] - 0s - loss: 0.4491 - acc: 0.7929 - val_loss: 0.4150 - val_acc: 0.8116
Epoch 58/100
623/623 [==============================] - 0s - loss: 0.4450 - acc: 0.7897 - val_loss: 0.4140 - val_acc: 0.8078
Epoch 59/100
623/623 [==============================] - 0s - loss: 0.4393 - acc: 0.8106 - val_loss: 0.4145 - val_acc: 0.8078
Epoch 60/100
623/623 [==============================] - 0s - loss: 0.4367 - acc: 0.8138 - val_loss: 0.4151 - val_acc: 0.8022
Epoch 61/100
623/623 [==============================] - 0s - loss: 0.4430 - acc: 0.8034 - val_loss: 0.4128 - val_acc: 0.8134
Epoch 62/100
623/623 [==============================] - 0s - loss: 0.4378 - acc: 0.8162 - val_loss: 0.4123 - val_acc: 0.8097
Epoch 63/100
623/623 [==============================] - 0s - loss: 0.4476 - acc: 0.8082 - val_loss: 0.4141 - val_acc: 0.7985
Epoch 64/100


623/623 [==============================] - 0s - loss: 0.4278 - acc: 0.8130 - val_loss: 0.4074 - val_acc: 0.8134
Epoch 87/100
623/623 [==============================] - 0s - loss: 0.4162 - acc: 0.8202 - val_loss: 0.4141 - val_acc: 0.8060
Epoch 88/100
623/623 [==============================] - 0s - loss: 0.4253 - acc: 0.8323 - val_loss: 0.4068 - val_acc: 0.8134
Epoch 89/100
623/623 [==============================] - 0s - loss: 0.4162 - acc: 0.8130 - val_loss: 0.4076 - val_acc: 0.8172
Epoch 90/100
623/623 [==============================] - 0s - loss: 0.4162 - acc: 0.8258 - val_loss: 0.4058 - val_acc: 0.8097
Epoch 91/100
623/623 [==============================] - 0s - loss: 0.4194 - acc: 0.8226 - val_loss: 0.4059 - val_acc: 0.8190
Epoch 92/100
623/623 [==============================] - 0s - loss: 0.4161 - acc: 0.8283 - val_loss: 0.4084 - val_acc: 0.8209
Epoch 93/100
623/623 [==============================] - 0s - loss: 0.4219 - acc: 0.8194 - val_loss: 0.4054 - val_acc: 0.8134
Epoch 94/100


In [94]:
# Load the weights that yielded the best validation accuracy
model.load_weights(model_weights_file)

# Evaluate the model on the training set
score = model.evaluate(X_train, y_train)
print("\nTraining Accuracy:", score[1])

736/891 [=======================>......] - ETA: 0s
Training Accuracy: 0.822109989646


In [95]:
# Load test data
df_test_raw = pd.read_csv(test_file)
print(df_test_raw.shape)
df_test_raw.head()
df_test_raw.info()

(418, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [96]:
# Prepare the data for testing
df_test = prep_data(df_test_raw)

# Construct the X array
X_test = np.array(df_test)[:,:]
X_test = X_test.astype('float32')
print(X_test.shape)
print(X_test[0])

Before scaling: 
   PassengerId  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  \
0          892       3    1  34.5      0      0   7.8292         1   
1          893       3    0  47.0      1      0   7.0000         2   
2          894       2    1  62.0      0      0   9.6875         1   
3          895       3    1  27.0      0      0   8.6625         2   
4          896       3    0  22.0      1      1  12.2875         2   

   Adult_Or_Minor  Senior_Citizen        ...         Ticket10  Ticket11  \
0               1               1        ...                0         0   
1               1               1        ...                0         0   
2               1               1        ...                0         0   
3               1               1        ...                0         0   
4               1               1        ...                0         0   

   Ticket12  Ticket13  Ticket14  Ticket15  Ticket16  Ticket17  In_Cabin  \
0         0         0         0     

In [97]:
# Predict for test data
y_test = model.predict(X_test)
print(y_test[0])

[ 0.88791049  0.11443626]


In [98]:
# Save predictions
with open(pred_file, 'w') as f:
    f.write('PassengerId,Survived\n')
    for index, y_hat in enumerate(y_test):
        prediction = np.argmax(y_hat)
        f.write(str(int(df_test_raw.iloc[index]['PassengerId'])) + ',' + str(prediction)+'\n')
    f.close()